In [2]:
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install bitsandbytes-cuda110 bitsandbytes
!pip install accelerate
!pip install langchain_community
!pip install PyPDF2
!pip install googletrans==3.1.0a0
!pip install arabic-reshaper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Get

In [3]:
import torch
from IPython.display import Markdown, display
from langchain import PromptTemplate
from langchain import HuggingFacePipeline
import PyPDF2
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
import PyPDF2
from arabic_reshaper import reshape
from googletrans import Translator


In [5]:
import httpcore
setattr(httpcore, 'SyncHTTPTransport', 'AsyncHTTPProxy')

In [6]:
translator = Translator()

In [7]:
translator.translate('فراتر از این کره ي خاکی', src='fa',dest='en').text

'Beyond this earth'

In [8]:
pdf_path = 'ebook8012[www.takbook.com].pdf'

In [36]:
with open(pdf_path, 'rb') as file:
      pdf_reader = PyPDF2.PdfReader(file)
      pages = pdf_reader.pages
      print(len(pages))
      text = ""
      en_text = ""
      for page in pages[10:40] :
        page_content = page.extract_text()+'\n'
        text+= page_content
        en_text += translator.translate(page_content,src='fa',dest='en').text

#
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)

141


In [23]:
part1 = text[:500]
while '\n \n' in part1:
  part1 = part1.replace('\n \n','')

In [25]:
print(part1)

      
  
  
  
  
  
  
  
  
  
  
  
  
 ﮔﺮد آوري : ﺳﻤﺎﻧﻪ ﻣﻌﺰي 
  
  
  
  
  
  
  
  
  
  
ﮐﯿﻬﺎن ﺷﻨﺎﺳﯽ ﻓﺮاﺗﺮ از اﯾﻦ ﮐﺮه ي 
ﺧﺎﮐﯽ  دﻓﺘﺮ اول از ﻣﺠﻤﻮﻋﻪ ي ﻋﻠﻤﯽ "دﻧﯿﺎي ﭘﯿﺮاﻣﻮن ﻣﺎ " www.takbook.com
  
دﻧﯿﺎي ﭘﯿﺮاﻣﻮن ﻣﺎ  
  
دﻓﺘﺮ اول:  
ﻓﺮاﺗﺮ از اﯾﻦ ﮐﺮه ي ﺧﺎﮐﯽ  
) ﮐﯿﻬﺎن ﺷﻨﺎﺳﯽ(  
  
ﮔﺮدآوري: ﺳﻤﺎﻧﻪ ﻣﻌﺰي    
www.takbook.com
  
ﺗﻘﺪﯾﻢ ﺑﻪ ﺗﻤﺎﻣﯽ دوﺳﺘﺪاران زﯾﺒﺎﯾﯽ  
  www.takbook.com
  
دﻧﯿﺎي ﭘﯿﺮاﻣﻮن ﻣﺎ  
دﻓﺘﺮ اول: ﻓﺮاﺗﺮ از اﯾﻦ ﮐﺮه ي ﺧﺎﮐﯽ  
ﮔﺮدآوري: ﺳﻤﺎﻧﻪ ﻣﻌﺰي  
زﻣﺴﺘﺎن 1392  
:اﯾﻤﯿﻞ  s.moezzi@vatan


In [17]:
print(text[:500].replace('\n\n',''))

 
 
   
 
  
  
  
  
  
  
  
  
  
  
  
  
  
 ﮔﺮد آوري : ﺳﻤﺎﻧﻪ ﻣﻌﺰي 
  
  
  
  
  
  
  
  
  
  
ﮐﯿﻬﺎن ﺷﻨﺎﺳﯽ ﻓﺮاﺗﺮ از اﯾﻦ ﮐﺮه ي 
ﺧﺎﮐﯽ  
 
دﻓﺘﺮ اول از ﻣﺠﻤﻮﻋﻪ ي ﻋﻠﻤﯽ "دﻧﯿﺎي ﭘﯿﺮاﻣﻮن ﻣﺎ " www.takbook.com
  
دﻧﯿﺎي ﭘﯿﺮاﻣﻮن ﻣﺎ  
  
دﻓﺘﺮ اول:  
ﻓﺮاﺗﺮ از اﯾﻦ ﮐﺮه ي ﺧﺎﮐﯽ  
) ﮐﯿﻬﺎن ﺷﻨﺎﺳﯽ(  
  
ﮔﺮدآوري: ﺳﻤﺎﻧﻪ ﻣﻌﺰي    
www.takbook.com
  
ﺗﻘﺪﯾﻢ ﺑﻪ ﺗﻤﺎﻣﯽ دوﺳﺘﺪاران زﯾﺒﺎﯾﯽ  
  www.takbook.com
  
دﻧﯿﺎي ﭘﯿﺮاﻣﻮن ﻣﺎ  
دﻓﺘﺮ اول: ﻓﺮاﺗﺮ از اﯾﻦ ﮐﺮه ي ﺧﺎﮐﯽ  
ﮔﺮدآوري: ﺳﻤﺎﻧﻪ ﻣﻌﺰي  
زﻣﺴﺘﺎن 1392  
:اﯾﻤﯿﻞ  s.moezzi@vatan


In [37]:
print(en_text[:500])

The world around us is cosmology 
 
 
 
5 | P a g e      
 Researcher: Samaneh Moazi Red giants, supernovae and neutron stars ..................................... ................................................ ....... 87  
Black hole ................................................... ................................................ ... 89  
In the center of their galaxy ................................................ ..... 90  
A stellar black hole ..........................................


In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [39]:
docs = text_splitter.split_text(en_text)

In [40]:
len(docs)

93

In [41]:
print(docs[0])

The world around us is cosmology 
 
 
 
5 | P a g e      
 Researcher: Samaneh Moazi Red giants, supernovae and neutron stars ..................................... ................................................ ....... 87  
Black hole ................................................... ................................................ ... 89  
In the center of their galaxy ................................................ ..... 90


In [42]:
print(docs[1])

A stellar black hole .................................................... ................................................ ..... 91  
Supermassive black holes .............................................. ................................................ ...... 91  
Black hole with average mass .................................................... ................................................ ...... 92


In [43]:
len(text.split())

6143

In [44]:

# ChromaDB requires us to have documents with two fields:
# page_content -- this is the document
# metadata -- not used, but we can use it for filtering if needed
class ChromaShip:
    def __init__(self, content, metadata=None):
        self.page_content: str = content
        self.metadata = metadata

# split the data into chunks, make sure the title is used with every chunk
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

documents = []
documents.extend([ChromaShip(content=doc, metadata=None) for doc in docs])


In [45]:
documents[0].page_content

'The world around us is cosmology \n \n \n \n5 | P a g e      \n Researcher: Samaneh Moazi Red giants, supernovae and neutron stars ..................................... ................................................ ....... 87  \nBlack hole ................................................... ................................................ ... 89  \nIn the center of their galaxy ................................................ ..... 90'

In [46]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

db = Chroma.from_documents(documents, embeddings, persist_directory="db_njscuba")

In [47]:
db.similarity_search_with_score("what is Solar System?",3)

[(Document(metadata={}, page_content='It is estimated that it is between 110 and 170 astronomical units from the sun.  \n     In the following, we will examine some of the objects of the solar system.  \n  \nwww.takbook.comThe world around us is cosmology \n \n \n \n01 | P a g e      \n Researcher: Samaneh Moazi 1- Khurshid \n \n     The sun is a star around which the earth and other objects of the solar system revolve. This body  \nDominating the solar system, it includes more than 99.8 percent of the mass of this system. Jam Khurshid 743'),
  0.24678475582126816),
 (Document(metadata={}, page_content='9 | P a g e      \n Researcher: Samaneh Moazi is the trailblazer. The solar system also has special regions, including the asteroid belt, \nKuiper and compact disc.  \n  \n The location of the solar system in the hunter arm, the Milky Way galaxy.  \n      There are thin and compressed substances called interplanetary medium between planets and other bodies. components \nConstruction of 

In [50]:
query_fa = "منظومه شمسی چیست"
query = translator.translate(query_fa,src='fa',dest='en').text
# result = translator.translate(result_en,src='en',dest='fa').text


result_en = '\n\n'.join(list(map(lambda x : x.page_content,db.similarity_search(query,3))))
result = translator.translate(result_en,src='en',dest='fa').text
display(Markdown(f"<b>{query_fa}</b>"))
display(Markdown(f"<p>{result}</p>"))

<b>منظومه شمسی چیست</b>

<p>یعنی به دور خورشید می چرخند. این سیستم شگفت انگیز الهام بخش اولین بحث در کیهان شناسی بود. 
تخصص دارد. جالب است بدانید این منظومه شمسی چگونه تشکیل شده و از چه ماده ای ساخته شده است. 
هست   
     منظومه شمسی از انفجار یک ابرنواختر و فروپاشی یک ابر چرخ متولد شد و در طی 
در دوران رنسانس و با مشاهدات افرادی از جمله گالیله گالیله کشف شد. این سیستم در بازوی شکارچی،

تخمین زده می شود که بین 110 تا 170 واحد نجومی از خورشید فاصله داشته باشد.  
     در ادامه به بررسی برخی از اجرام منظومه شمسی می پردازیم.  
  
www.takbook.com دنیای اطراف ما کیهان شناسی است 
 
 
 
01 | P a g e      
 محقق: سمانه معزی 1- خورشید 
 
     خورشید ستاره ای است که زمین و سایر اجرام منظومه شمسی به دور آن می چرخند. این بدن  
با تسلط بر منظومه شمسی، بیش از 99.8 درصد از جرم این منظومه را شامل می شود. جام خورشید ۷۴۳

9 | P a g e      
 محقق: سمانه معزی پیشرو است. منظومه شمسی همچنین دارای مناطق ویژه ای از جمله کمربند سیارکی است. 
کویپر و دیسک فشرده.  
  
 مکان منظومه شمسی در بازوی شکارچی، کهکشان راه شیری.  
      بین سیارات و اجرام دیگر مواد نازک و فشرده ای به نام محیط بین سیاره ای وجود دارد. اجزاء 
ساخت محیط هیدروژن خنثی و غیر یونیزه، گاز پلاسما، پرتوهای کیهانی و ذرات</p>

In [ ]:
!pip install bitsandbytes-cuda110 bitsandbytes


In [ ]:
!pip install accelerate

restart

In [ ]:
# Set up Mistral7B
# resource link: https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# specify the llm


In [ ]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

In [ ]:
response = qa_chain.run(
    "what is “Backpropagation”?")
print(response)

In [ ]:
query_fa = "منظومه شمسی چیست"
query = translator.translate(query_fa,src='fa',dest='en').text
result_en = llm(query)
result = translator.translate(result_en,src='en',dest='fa').text

display(Markdown(f"<b>{query_fa}</b>"))
display(Markdown(f"<p>{result}</p>"))